### Coordinate descent algorithm


In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
# Define soft-threshoding

def threshold(s,lam):
    
    """
    s: l2 norm of the ith row of either X.T@U or XV
    lam: lambda_u or lambda_v(s)
    
    """
    
    if s <= lam:
        return 0
    else:
        return 1-lam/s

In [39]:
def ridge(s,lam):
    
    """
    Ridge penalty
    """
    
    return s/(1+lam)

In [40]:
def inner_ridge(X,lam):
    n = X.shape[0]
    
    for i in range(n):
        X[i,:] = X[i,:]/(1+lam)
        
    return X

In [84]:
# Get initial valuse by regular SVD
def initial_svd(X,X0,k=3, lamv=[0.05]*D,lamu = 0.05):
    
    U0, s0, V0t = np.linalg.svd(X0)
    U0 = U0[:,0:k]
    svdeach = list(map(lambda x: np.linalg.svd(x), X))
    V0 = list(map(lambda x: x[2].T[:,0:k], svdeach))
    
    return U0,s0,V0

In [3]:
def cost_func(X,U,V,lamv=[0.05]*3,lamu=0.05):
    
    costv = [lamv[i]*np.sum(np.linalg.norm(V[i],2,axis=1)) for i in range(len(V))]
    costu = lamu*np.sum(np.linalg.norm(U,2,axis = 1))
    V = np.concatenate(V,axis = 0)
    cost = np.linalg.norm((X-U@V.T),'fro') + np.sum(costv)+ costu
    
    return cost

In [2]:
def inner_cd(X,lam):
    
    n,k = X.shape
    X1 = np.zeros([n,k])
    
    for j in range(n):
        xj_norm = np.linalg.norm(X[j,:],2)
        if xj_norm <= lam:
            X1[j,:] = np.zeros([1,k])
        else:
            X1[j,:] = (1-lam/xj_norm) * X[j,:]
        
    #W1 = W / np.sqrt(np.sum(W**2, axis=0))
    
    
    return X1
        

In [3]:
"""
Beging testing the algorithm at this point

"""
# n=50 ; p = 100
#np.random.seed(1)
#Xdata = np.random.normal(size = (n,p))
Xdata = pd.read_csv("Xdata_from_r.csv")
del Xdata['Unnamed: 0']

In [4]:
scaler = StandardScaler()
Xdata_scaled = scaler.fit_transform(Xdata)

In [5]:
"""
Initialization

"""
U0, s0, V0t = np.linalg.svd(Xdata_scaled)
k = 3; lamv = 0.1
U0 = U0[:,0:k]
V0 = V0t.T[:,0:k]

In [128]:
"""
Test w.r.t. V matrix

First, thresholding with sj = (X.t@U)j
"""

S0 = Xdata_scaled.T @ U0
#S0 = V0

i=0
for r in range(100):

    i+=1
    
    S1 = inner_cd(S0,lamv)
    
    Sd = np.linalg.norm(S0-S1,2)
    
    if Sd <= 1e-7:
        break

    S0 = S1



In [25]:
S0 = Xdata_scaled.T @ U0

In [38]:
S1 = inner_cd(S0,lamv)

In [39]:
S1

array([[-1.8902083 ,  0.45424022,  2.97390777],
       [-1.09087154, -0.49313906, -2.08529238],
       [-0.57072419,  0.79221088, -1.21444812],
       [ 0.60176625, -0.38008233, -1.97309138],
       [-0.44477707,  1.07247868,  0.64330427],
       [-1.0286475 , -1.29838138, -0.36063068],
       [-0.73890669, -0.26842276,  1.08642185],
       [ 2.07237397, -0.97997347, -0.09196375],
       [ 0.00971903, -0.22899203, -0.4880624 ],
       [ 0.68597594,  0.0436844 , -3.7317324 ],
       [ 0.34681126, -0.99599458, -0.70148215],
       [-1.86670777, -1.64431748,  0.02257244],
       [-1.31076995, -1.83710056,  1.59334637],
       [-1.92207648,  1.36094772, -1.13121528],
       [-1.19825742, -1.77609434,  0.14813487],
       [ 1.71214375,  0.06428659,  1.93740412],
       [ 0.43870135, -2.04166078,  2.11983527],
       [ 1.16777992, -1.65878531, -1.53574582],
       [ 0.15434453, -1.74975258, -0.67468488],
       [ 4.01418597,  0.38718715,  0.18118992],
       [-1.34580177, -0.16755849, -1.789

In [34]:
X1 = U0 @ S1.T

In [35]:
Sd = np.linalg.norm(S0-S1,2)

In [36]:
Sd

0.5949919786890964

In [37]:
S0 = X1.T @ U0 

In [15]:
S1_or = np.linalg.qr(S1)[0]

In [ ]:
S1_or

In [17]:
V0 = S1_or

In [22]:
Z0 = Xdata_scaled @ V0

In [19]:
lamu = 0.1

In [24]:
i=0
for r in range(100):
    
    i+=1
    
    Z1 = inner_cd(Z0,lamu)
    
    Zd = np.linalg.norm(Z0-Z1,2)
    
    if Zd <= 1e-7:
        break

    Z0 = Z1

In [25]:
Z0

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0

In [78]:
a = 3
b = 5
test(a,b)

9

In [79]:
b

5

In [70]:
S0

array([[-1.99661093,  0.47981008,  3.14131345],
       [-1.18160729, -0.53415704, -2.25874137],
       [-0.64397514,  0.89388907, -1.37031936],
       [ 0.65914453, -0.4163231 , -2.16122521],
       [-0.511794  ,  1.23407475,  0.74023435],
       [-1.150002  , -1.45155768, -0.40317602],
       [-0.84910703, -0.30845525,  1.2484505 ],
       [ 2.2530328 , -1.06540248, -0.09998067],
       [ 0.013324  , -0.31392972, -0.66909443],
       [ 0.72213218,  0.0459869 , -3.92842359],
       [ 0.40157237, -1.153261  , -0.81224539],
       [-2.0167799 , -1.77651076,  0.02438713],
       [-1.40566481, -1.97009981,  1.70869872],
       [-2.06920979,  1.4651271 , -1.21780884],
       [-1.3098467 , -1.94149543,  0.16193012],
       [ 1.84454322,  0.06925784,  2.0872229 ],
       [ 0.46818736, -2.17888495,  2.2623137 ],
       [ 1.25957334, -1.78917425, -1.65646322],
       [ 0.17074965, -1.93573195, -0.74639643],
       [ 4.21306143,  0.40636962,  0.19016664],
       [-1.46566235, -0.18248168, -1.949

In [38]:
S0 = Xdata_scaled.T @ U0

In [40]:
np.linalg.norm(S0-S1,2)

1.189983957378193

In [22]:
"""
Also, thresholding with sj = V0_j itself
"""

inner_cd(V0,lamv)

array([[ 0., -0., -0.],
       [-0.,  0.,  0.],
       [ 0., -0., -0.],
       [-0., -0.,  0.],
       [ 0., -0., -0.],
       [-0.,  0., -0.],
       [ 0.,  0., -0.],
       [-0., -0., -0.],
       [-0., -0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0., -0.],
       [-0.,  0.,  0.],
       [-0., -0., -0.],
       [-0., -0.,  0.],
       [ 0., -0.,  0.],
       [-0., -0., -0.],
       [ 0., -0.,  0.],
       [-0., -0., -0.],
       [-0., -0., -0.],
       [-0., -0., -0.],
       [ 0.,  0.,  0.],
       [ 0., -0., -0.],
       [ 0., -0.,  0.],
       [-0.,  0., -0.],
       [ 0., -0.,  0.],
       [-0.,  0., -0.],
       [ 0., -0.,  0.],
       [ 0., -0.,  0.],
       [ 0.,  0., -0.],
       [ 0.,  0., -0.],
       [ 0.,  0.,  0.],
       [ 0., -0., -0.],
       [ 0., -0., -0.],
       [-0.,  0., -0.],
       [-0.,  0.,  0.],
       [ 0., -0.,  0.],
       [ 0.,  0.,  0.],
       [-0., -0.,  0.],
       [-0., -0.,  0.],
       [-0., -0.,  0.],
       [-0., -0., -0.],
       [ 0.,  0.

In [206]:
def coordinate_descent(U0,V0,X,X0,lamv=[0.05]*D,lamu = 0.05,iters = 100,eps=1e-3):
    
#     cost_history=[]
#     cost0 = cost_func(X0,U0,V0,lamv,lamu)
#     cost_history.append(cost0)
    D = len(X)
    ud = [1]
    vd = [[1,1,1]]
    maxdist = 1
    r=0
#     V0t = np.concatenate(V0,axis=0)
#     Xr0 = U0 @ V0t.T
    
    while maxdist>eps and r<=iters:
        r+=1
        # Update V
        d = -1
        V1 = []
        tempv1 = []
        for Xd in X:
            d += 1
            V2 = V0[d]
            Sd = Xd.T @ U0
            tempv = inner_cd(V2,Sd,lamv[d])
            tempv1.append(tempv)
            
            V1.append(tempv / np.sqrt(np.sum(tempv**2, axis=0)))
        
            
        # Updata U
        
        Vu = np.concatenate(V0,axis=0)
        Zd = X0@Vu
        tempu1 = inner_cd(U0,Zd,lamu)
        u1 = tempu1 / np.sqrt(np.sum(tempu1**2, axis=0))
        
        ud.append(np.linalg.norm(U0-U1,2))
        vd.append([np.linalg.norm(V0[i]-V1[i],2) for i in range(D)])
        
        U0 = tempu1
        V0 = tempv1
        
#         # convergence by reconstructing
#         Xr1 = U0@Vu.T
#         maxdist = np.linalg.norm(Xr0-Xr1,2)
        
        
        vm1 = [np.max(vd[i]) for i in range(len(vd))]
        vm2 = np.max(vm)
        um = np.max(ud)
        maxdist = max(um,vm2)
#         cost_history.append(cost_func(X0,U0,V0,lamv,lamu))
        
    
    return U0,V0,maxdist
            
    

In [ ]:
# Example data
n=50
p = [70,80,100]
X = [np.random.normal(size=(n,i)) for i in p]

In [207]:
# Initialization

X0 = np.concatenate(X,axis = 1)
D = len(X)

lamv = [0.05]*D
lamu = 0.05

U0,s0,V0 = initial_svd(X=X,X0=X0,k=3,lamv=lamv,lamu=lamu)


eps = 0.001

# Updating

Un,Vn,maxdist = coordinate_descent(U0=U0,V0=V0,X=X,X0=X0,lamv=lamv,lamu=lamu,iters=100, eps=1e-3)

In [227]:
U0,s0,V0 = initial_svd(X=X,X0=X0,k=3,lamv=lamv,lamu=lamu)

In [231]:
V0 = np.concatenate(V0,axis=0)

In [224]:
import glmnet_python as glm

In [266]:
import glmnet_python
from glmnet import glmnet
import scipy as sp

In [241]:
y = np.random.normal(size=(10,20))
u0,s0,v0t = np.linalg.svd(y)
u0 = u0[:,0:3]
v0 = v0t.T[:,0:3]

In [264]:
# load data from r example
x = pd.read_csv("xdata.csv")
x = x.iloc[:,1:]
y = pd.read_csv("ydata.csv")
y = y.iloc[:,1:]

(100, 4)

In [270]:
x = sp.array(x,dtype=sp.float64)
y = sp.array(y,dtype=sp.float64)

In [271]:
mfit = glmnet(x = x, y = y, family = 'mgaussian')

ValueError: loadGlmlib does not currently work for windows

In [ ]:
d = -1
for Xd in X:
    d += 1
    glm.glmnet()